In [2]:
import os, glob
import json, pickle
import nltk
import re
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer

from collections import Counter
import tqdm

In [2]:
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('tagsets')
# nltk.download('stopword')
# nltk.download('maxent_treebank_pos_tagger')
# nltk.download('averaged_perceptron_tagger')

In [3]:
os.getcwd()

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\airbnb-NLP'

In [5]:
os.chdir('./../airbnb-data')

In [6]:
file_lst = glob.glob('translated_*.json')
file_lst

['translated_busan.json',
 'translated_daegu.json',
 'translated_daejeon.json',
 'translated_gangneung.json',
 'translated_gwangju.json',
 'translated_jeju.json',
 'translated_jeonju.json',
 'translated_seogwipo.json',
 'translated_seoul.json',
 'translated_seoul1.json',
 'translated_seoul2.json',
 'translated_seoul3.json',
 'translated_yeosu.json']

In [109]:
review = {}
for file in file_lst:
    with open(file,'r',encoding='utf-8') as fp:
        review.update(json.load(fp))

In [110]:
len(review)

32957

In [111]:
review['10030136'][0]

{'collection_tag': None,
 'comments': '친구들이랑 6명이 갔는데\n숙소 위치도 너무 좋고 방도 깨끗해서\n다들 맘에 들어했어요!!\n에어비엔비를  처음 이용해봤는데\n너무 좋아서 다음번에도 이용할것같아요\n해운대해변이랑도 걸어갈수 있고\n연락도 너무 잘됐어요',
 'created_at': '2016-10-08T23:57:42Z',
 'id': 106917581,
 'language': 'ko_en',
 'localized_date': '2016년 10월',
 'rating': 5,
 'response': '',
 'reviewee': {'deleted': False,
  'first_name': 'Jaewook',
  'host_name': 'Jaewook',
  'id': 38530724,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/users/38530724/profile_pic/1436955527/original.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/38530724'},
 'reviewer': {'deleted': False,
  'first_name': 'Joohee',
  'host_name': 'Joohee',
  'id': 96535213,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/pictures/32cbca58-85f2-47e5-b09c-a5ead7662304.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/96535213'},
 't_comments': 'I went to the dormitory with my friends. They liked the room because it was so nice and

In [112]:
review_en = {}
for home_id,rev_lst in review.items():
    review_en[home_id] = []
    for rev in rev_lst:
        if  rev['language'] is not None and rev['language'][-3:]=='_en':
            review_en[home_id].append([rev['rating'],rev['t_comments']])
        else:
            continue

In [130]:
review_en['10030136']

[[5,
  'I went to the dormitory with my friends. They liked the room because it was so nice and clean!This is my first time using Air Bienby. I love it, so I think I will use it next time.I can walk to Haeundae beach and the contact was great.'],
 [5,
  "The room was spacious and spacious, and the two bedrooms could be seen from the window.When we went there, we caught the typhoon and fortunately the room was occupied with double-decked windows.The room is modern and technological.Overall, it's pretty good."],
 [5,
  "When I arrived, my family praised me for my clean and spacious accommodation! The front was a construction site, so the view wasn't 100 percent cool, but it was enough to last.Ha"],
 [5,
  "The room is prettier and more sunny than the one you see in the picture. Especially, Haeundae Beach looks so peaceful, with two harmonics in the living room brightening the atmosphere of the interior.Thanks to the host's consideration, I could take a comfortable trip to Busan.Thank you

In [96]:
## spacing 필요하지않을까?? 문장 마침표 후 띄어쓰기 안된 경우 보정하자 

In [127]:
tmp_sent = 'I went to the dormitory with my friends.They liked the room because it was so nice and clean!This is my first time using Air Bienby.I love it, so I think I will use it next time.I can walk to Haeundae beach and the contact was great.'

In [128]:
txt = tmp_sent
txt = re.sub('[^a-z|A-Z|0-9|.!?,\')(|\s]','',txt.strip())
txt = re.sub( r'([a-zA-Z])([.!?])([a-zA-Z])', r'\1\2 \3', txt)

In [129]:
txt

'I went to the dormitory with my friends. They liked the room because it was so nice and clean! This is my first time using Air Bienby. I love it, so I think I will use it next time. I can walk to Haeundae beach and the contact was great.'

In [97]:
re.sub('[^a-z|A-Z|0-9|.!?,\')(|\s]','','asdlk. fj!          a(sd)fㅁㄴ 아\'as\"d  23ㅁㄴㅇ!!@as')

"asdlk. fj!          a(sd)f 'asd  23!!as"

In [131]:
for k,rev_lst in review_en.items():
    for i,rev in enumerate(rev_lst):
        txt = rev[1]
        txt = re.sub('[^a-z|A-Z|0-9|.!?,\')(|\s]','',txt.strip())
        txt = re.sub( r'([a-zA-Z])([.!?])([a-zA-Z])', r'\1\2 \3', txt)
        rev_lst[i][1] = txt

In [99]:
# breakpoint = 0
# for k,rev_lst in review_en.items():
#     for rev in rev_lst:
#         if breakpoint == 1000:
#             break
#         print(rev[1],end='\n\n')
#         breakpoint+=1

In [132]:
sent_tokenize(review_en['10030136'][1][1]) # sent_tokenize 안되는 리뷰 꽤 있음 -> 해결

['The room was spacious and spacious, and the two bedrooms could be seen from the window.',
 'When we went there, we caught the typhoon and fortunately the room was occupied with doubledecked windows.',
 'The room is modern and technological.',
 "Overall, it's pretty good."]

In [133]:
%%time
# 리뷰별 문장단위 tokenizing
for home_id,rev_lst in review_en.items():
    for rev in rev_lst:
        rev[1] = sent_tokenize(rev[1])

Wall time: 1min 20s


In [ ]:
# 이후 부적절한 길이의 문장 제거

In [147]:
for home_id,rev_lst in review_en.items():
    for i,rev in enumerate(rev_lst):
        tmp_rev = []
        for sent in rev[1]:
            if len(sent) < 5:
                continue
            else:
                tmp_rev.append(sent)
        rev_lst[i][1] = tmp_rev

In [148]:
%%time 
# 문장별 단어단위 tokenizing
for home_id,rev_lst in review_en.items(): 
    for rev in rev_lst:
        tmp_lst = []
        for sent in rev[1]:
            word_lst = nltk.word_tokenize(sent)
            tmp_lst.append(word_lst)
        rev[1] = tmp_lst

Wall time: 5min 32s


In [149]:
nltk.pos_tag(review_en['10030136'][1][1][0])

[('The', 'DT'),
 ('room', 'NN'),
 ('was', 'VBD'),
 ('spacious', 'JJ'),
 ('and', 'CC'),
 ('spacious', 'JJ'),
 (',', ','),
 ('and', 'CC'),
 ('the', 'DT'),
 ('two', 'CD'),
 ('bedrooms', 'NNS'),
 ('could', 'MD'),
 ('be', 'VB'),
 ('seen', 'VBN'),
 ('from', 'IN'),
 ('the', 'DT'),
 ('window', 'NN'),
 ('.', '.')]

In [150]:
k_to_drop = []
for k,rev_lst in review_en.items():
    if review_en[k] == []:
        k_to_drop.append(k)

In [151]:
for k in k_to_drop:
    del review_en[k]

In [152]:
len(review_en) # 리뷰없는 방 제거 후 

22886

In [183]:
for home_id,rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        tmp_lst = []
        for sent_bow in rev[1]:
            sent_pos = nltk.pos_tag(sent_bow)
            tmp_lst.append(sent_pos)
        rev[1] = tmp_lst

### 데이터 구조를 보자

- review_en == {home_id: [rev0, rev1, ...]}
    - rev0 == [rating, [sent0, sent1, ...]]
        - sent0 == [(word0,POS0),(word1,POS1), ...]
    

In [154]:
# with open('review_en_pos_0113_2.dic','wb') as fp:
#     pickle.dump(review_en,fp)

In [155]:
review_en['10022410'][0] # rev0의 예시

[5,
 [[('Very', 'RB'), ('good', 'JJ'), ('.', '.')],
  [('The', 'DT'),
   ('landlord', 'NN'),
   ('is', 'VBZ'),
   ('very', 'RB'),
   ('enthusiastic', 'JJ'),
   ('.', '.')],
  [('It', 'PRP'),
   ("'s", 'VBZ'),
   ('a', 'DT'),
   ('little', 'JJ'),
   ('bug', 'NN'),
   ('in', 'IN'),
   ('the', 'DT'),
   ('house', 'NN'),
   ('.', '.')]]]

unpacking과 zip 함수 이용해 rating과 review를 각각을 뽑아 리스트로 만들 수도 있다

In [156]:
rating_lst, rev_lst = zip(*review_en['10022410'])

In [157]:
rating_lst

(5, 5, 5, 5, 3, 5, 2, 5, 5, 2, 5, 5, 5, 4, 5, 5, 3, 4, 5, 5, 4, 4, 5)

In [158]:
rev_lst[0]

[[('Very', 'RB'), ('good', 'JJ'), ('.', '.')],
 [('The', 'DT'),
  ('landlord', 'NN'),
  ('is', 'VBZ'),
  ('very', 'RB'),
  ('enthusiastic', 'JJ'),
  ('.', '.')],
 [('It', 'PRP'),
  ("'s", 'VBZ'),
  ('a', 'DT'),
  ('little', 'JJ'),
  ('bug', 'NN'),
  ('in', 'IN'),
  ('the', 'DT'),
  ('house', 'NN'),
  ('.', '.')]]

In [159]:
ratings = []
review_wo_ratings = []
for k,rev_lst in review_en.items():
    tmp_rating, tmp_rev = zip(*review_en[k])
    ratings += tmp_rating
    review_wo_ratings += tmp_rev

In [160]:
len(ratings)

383823

In [161]:
len(review_wo_ratings)

383823

In [162]:
# with open('ratings_0113.lst','wb') as fp:
#     pickle.dump(ratings,fp)

In [18]:
# with open('pos_lst_review.json','w') as fp:
#     json.dump(review_en,fp)

In [179]:
with open('review_en_pos_0113_2.dic','rb') as fp:
    review_en = pickle.load(fp)

고유명사 제외한다

In [163]:
NNP_lst = []
for k,rev_lst in review_en.items():
    for rev in rev_lst:
        for sent in rev[1]:
            for w in sent:
                if w[1] == 'NNP' or w[1] == 'NNPS':
                    NNP_lst.append(w)

In [164]:
len(NNP_lst)

426427

In [166]:
sorted(Counter(NNP_lst).items(),key=lambda kv:kv[1],reverse=True)[:30]

[(('Thank', 'NNP'), 38231),
 (('Jeju', 'NNP'), 21443),
 (('Station', 'NNP'), 14392),
 (('Seoul', 'NNP'), 11541),
 (('Busan', 'NNP'), 10927),
 (('Island', 'NNP'), 9921),
 (('First', 'NNP'), 7133),
 (('Haeundae', 'NNP'), 6802),
 (('Airbnb', 'NNP'), 6347),
 (('Village', 'NNP'), 4290),
 (('Korea', 'NNP'), 4251),
 (('Hanok', 'NNP'), 4243),
 (('Beach', 'NNP'), 3895),
 (('Please', 'NNP'), 3468),
 (('Daegu', 'NNP'), 3062),
 (('Lotte', 'NNP'), 2856),
 (('Jeonju', 'NNP'), 2619),
 (('Gangnam', 'NNP'), 2588),
 (('University', 'NNP'), 2547),
 (('Hongdae', 'NNP'), 2464),
 (('Gangneung', 'NNP'), 2211),
 (('Yeosu', 'NNP'), 2185),
 (('Seongsan', 'NNP'), 2141),
 (('Nice', 'NNP'), 2108),
 (('Myeongdong', 'NNP'), 2021),
 (('Host', 'NNP'), 2019),
 (('Itaewon', 'NNP'), 1926),
 (('Gwangalli', 'NNP'), 1920),
 (('Park', 'NNP'), 1913),
 (('Clean', 'NNP'), 1901)]

In [ ]:
#NNP 제거?? LDA 할때 제거

In [167]:
NNP_to_leave = ['Thank','Station','Island','Hanok','Village','University','Beach','Please','Nice','Bridge','City','Market','WiFi','Host','Breakfast','Mart','English','Bedding','House','River','World','View','Airport','Recommend','Tower','Korean'
                ,'Line','Stay','Korean','Store','Lake','Okay','Port','Bluetooth','Kind'
                ,'Exit','Peak','Ocean','Hotel','Chinese','Sea','Hot','Hidden','Christmas','Internet','Starbucks','Water','Cafe','Home','Bus','Take','Center','Room'
                ,'Good','Hall','Great','Amenity','Subway','Museum','International','Rest','Clean']

In [168]:
for k,rev_lst in review_en.items():
    for rev in rev_lst:
        for i,sent in enumerate(rev[1]):
            tmp_sent = []
            for word in sent:
                if word[1] == 'NNP' and word[0] not in NNP_to_leave:
                    continue # drop this
                else:
                    tmp_sent.append(word)
            rev[1][i] = tmp_sent
                    
                    

TF-IDF 적용해 중요한 단어만 남긴다

In [56]:
# words = []
# for w, rev_lst in review_en.items():
#     for rev in rev_lst:
#         for sent in rev[1]:
#             for w in sent:
#                 if w[1] == 'NNP' or w[1] == 'NNPS':
#                     continue
#                 else:
#                     words.append(w)

In [59]:
# ws,tags = zip(*words)

In [60]:
# unq_words = set(ws)

In [61]:
# len(unq_words)

175632

In [62]:
# doc_tokens = {}
# maxCount = {}

# for i,lst in enumerate(unq_words):
#     maxCount[i] = 0
#     tmp_tokens = {}
    
#     for w in lst:
#         if w in tmp_tokens.keys():
#             tmp_tokens[w] += 1
#         else:
#             tmp_tokens[w] = 1

#         if maxCount[i] < tmp_tokens[w]:
#             maxCount[i] = tmp_tokens[w]
                    
#     doc_tokens[i] = tmp_tokens


명사, 형용사, 동사, 부사를 뽑아 lemmantization 시켜주자

함수 njvr_lemmantizer를 정의한다


In [169]:
lemm = WordNetLemmatizer()
def njvr_lemmantizer(sent):
    global lemm
    
    lemm_sent = []
    for word_pos in sent:
        word,pos=word_pos
        if pos[0] == 'N':
            lemm_sent.append(lemm.lemmatize(word,pos='n'))
        elif pos[0] == 'J':
            lemm_sent.append(lemm.lemmatize(word,pos='a'))
        elif pos[0] == 'V':
            lemm_sent.append(lemm.lemmatize(word,pos='v'))
        elif pos[0] == 'R':
            lemm_sent.append(lemm.lemmatize(word,pos='r'))
        else:pass
    return lemm_sent

샘플 sentence

In [170]:
review_en['10022410'][1][1][0]

[('The', 'DT'),
 ('space', 'NN'),
 ('was', 'VBD'),
 ('very', 'RB'),
 ('clean', 'JJ'),
 ('and', 'CC'),
 ('the', 'DT'),
 ('view', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('sea', 'NN'),
 ('was', 'VBD'),
 ('very', 'RB'),
 ('good', 'JJ'),
 ('.', '.')]

lemmantization 적용후

In [171]:
njvr_lemmantizer(review_en['10022410'][1][1][0])

['space', 'be', 'very', 'clean', 'view', 'sea', 'be', 'very', 'good']

전체 리뷰에 적용하자

In [184]:
for listing_id, rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        rev_tmp = []
        for sent in rev[1]:
            rev_tmp.append(njvr_lemmantizer(sent))
        rev[1] = rev_tmp

In [173]:
list(review_en.items())[9][1][1]

[5,
 [['be', 'second', 'stay', 'hearty', 'breakfast', 'be', 'definitely', 'great'],
  ['Other',
   'like',
   'visit',
   'store',
   'visit',
   'doctor',
   'Chinese',
   'medicine',
   'again']]]

to_lower

In [185]:
for listing_id, rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        for i,sent in enumerate(rev[1]):
            rev[1][i] = list(map(str.lower,sent))

In [182]:
with open('review_NJVR_with_NNP_0113.json','w') as fp:
    json.dump(review_en,fp)